<a href="https://colab.research.google.com/github/ctarrington/data-science-hw/blob/master/kalman-filters/rlabbe/rlabbe_01_g_h_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def g_h_filter(zs, x0, x_dot0, g, h, dt):
  x = x0
  x_dot = x_dot0

  estimates = []
  for z in zs:
    predicted_x = x + x_dot * dt

    residual = z - predicted_x
    x = predicted_x + g * residual
    x_dot = x_dot + h * residual / dt

    estimates.append(x)

  return np.array(estimates)



In [3]:
weights = [158.0, 164.2, 160.3, 159.9, 162.1, 164.6, 
           169.6, 167.4, 166.4, 171.0, 171.2, 172.6]

In [4]:
estimates = g_h_filter(weights, 160, 1, .6, 2./3, 1)

In [5]:
print(estimates)

[159.2        161.8        162.1        160.78       160.98533333
 163.31057778 168.10029037 169.69598183 168.20360386 169.1642501
 170.89234186 172.62868399]
